<a href="https://colab.research.google.com/github/aleeepassarelli/scientific-validation-hub/blob/main/notebooks/ACC_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Semantic Density](https://img.shields.io/badge/Semantic_Density-PASS-brightgreen)


In [ ]:
name_to_test = "explorador api"
domain_to_test = "explorador api"

validation_results = run_validation(name_to_test, domain_to_test)

print("\n--- Resultados da Validação ---")
print(f"Nome: {name_to_test}")
print(f"Domínio: {domain_to_test}")

for key, value in validation_results.items():
    if key == 'results_per_model':
        print(f"  {key.replace('_', ' ').title()}:")
        for model_key, model_value in value.items():
            print(f"    Modelo {model_key}: SD = {model_value['sd']:.4f}")
    elif key == 'thresholds':
        print(f"  {key.replace('_', ' ').title()}:")
        for threshold_key, threshold_value in value.items():
            print(f"    {threshold_key.replace('_', ' ').title()}: {threshold_value}")
    else:
        print(f"  {key.replace('_', ' ').title()}: {value}")



🔬 VALIDANDO: explorador api | Domínio: explorador api

--- Resultados da Validação ---
Nome: explorador api
Domínio: explorador api
  Status Sd: PASS
  Status Minimalism: PASS
  Sd Mean: 0.9999999701976776
  Sd Min: 0.9999999403953552
  Word Count: 2
  Results Per Model:
    Modelo miniLM: SD = 1.0000
    Modelo mpnet: SD = 1.0000
  Thresholds:
    Sd Pass: 0.7
    Sd Cross Platform: 0.55
    Minimalism: 3


In [ ]:
# validation_core.py
# Versão Minimalista para Notebook Colab
# Implementa SD (Densidade Semântica) e Minimalismo.

from sentence_transformers import SentenceTransformer, util
import time
from typing import Tuple, Dict, Any

# --- Constantes de Validação (A "Ciência" do ACC) ---
# Removendo o modelo 'multilingual' conforme solicitado
EMBEDDING_MODELS = {
    'miniLM': 'all-MiniLM-L6-v2',  # Leve, rápido
    'mpnet': 'all-mpnet-base-v2'  # Balanceado
}
THRESHOLD_PASS = 0.70  # Threshold principal para SD (Cosine Sim)
THRESHOLD_MIN_CROSS_PLATFORM = 0.55 # Mínimo aceitável em benchmark
THRESHOLD_MINIMALISM = 3 # Número máximo de palavras no Nome do Agente

# Armazenamento em cache para modelos já carregados
_LOADED_MODELS = {}

def load_models() -> Dict[str, SentenceTransformer]:
    """
    Carrega todos os modelos de embedding UMA VEZ e os armazena em cache.
    """
    global _LOADED_MODELS
    if _LOADED_MODELS and len(_LOADED_MODELS) == len(EMBEDDING_MODELS):
        return _LOADED_MODELS

    # Limpar modelos carregados se a configuração EMBEDDING_MODELS mudou
    _LOADED_MODELS.clear()

    print(f"⏳ Carregando {len(EMBEDDING_MODELS)} modelos de embedding...")
    start_load = time.time()

    try:
        for key, model_name in EMBEDDING_MODELS.items():
            _LOADED_MODELS[key] = SentenceTransformer(model_name)
    except Exception as e:
        # No Colab, erros fatais devem ser lançados para a célula falhar.
        raise RuntimeError(f"❌ ERRO FATAL: Falha ao carregar modelos: {e}")

    load_time = time.time() - start_load
    print(f"✅ Modelos carregados e prontos em {load_time:.2f}s.")
    return _LOADED_MODELS

def calculate_sd(name: str, domain: str, model: SentenceTransformer) -> Tuple[float, int]:
    """
    Calcula a Densidade Semântica (SD) e a contagem de palavras.
    """
    embed_name = model.encode(name, convert_to_tensor=True)
    embed_domain = model.encode(domain, convert_to_tensor=True)

    # SD é a Similaridade de Cossenos.
    sd = util.cos_sim(embed_name, embed_domain)[0][0].item()
    word_count = len(name.split())

    return sd, word_count

def run_validation(name: str, domain: str) -> Dict[str, Any]:
    """
    Executa a validação multi-modelo e retorna um dicionário consolidado.
    """
    loaded_models = load_models()
    results = {}

    print(f"\n{'='*70}")
    print(f"🔬 VALIDANDO: {name} | Domínio: {domain}")
    print(f"{'='*70}")

    word_count = 0
    sd_values = []

    for key, model in loaded_models.items():
        sd, wc = calculate_sd(name, domain, model)
        if word_count == 0:
            word_count = wc

        results[key] = {'sd': sd}
        sd_values.append(sd)

    # --- Veredito Final (Lógica de Dupla Condição) ---
    sd_mean = sum(sd_values) / len(sd_values)
    sd_min = min(sd_values)

    # 1. Teste de Densidade Semântica
    pass_sd = sd_mean >= THRESHOLD_PASS and sd_min >= THRESHOLD_MIN_CROSS_PLATFORM

    # 2. Teste de Minimalismo
    pass_minimalism = word_count <= THRESHOLD_MINIMALISM

    # Retorna o dicionário de resultados e status
    return {
        'status_sd': 'PASS' if pass_sd else 'FAIL',
        'status_minimalism': 'PASS' if pass_minimalism else 'FAIL',
        'sd_mean': sd_mean,
        'sd_min': sd_min,
        'word_count': word_count,
        'results_per_model': results,
        'thresholds': {
            'sd_pass': THRESHOLD_PASS,
            'sd_cross_platform': THRESHOLD_MIN_CROSS_PLATFORM,
            'minimalism': THRESHOLD_MINIMALISM
        }
    }

# Fim do Arquivo validation_core.py
